In [43]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
from playground.load import dataset, clip_model, clip_processor, model_device, extract_image_vectors

In [44]:
# will take a few minutes
# df, image_vectors = extract_image_vectors(dataset)
# df.to_parquet('data/image_meta.parquet')
# np.save('data/image_vectors.npy', image_vectors)
df = pd.read_parquet('data/objectnet/image_meta.parquet')
image_vectors = np.load('data/objectnet/image_vectors.npy')
image_vectors = image_vectors

In [3]:
def get_y_true(df, label):
    assert label in df.label.unique()
    return np.where(df.label == label, 1, 0)

def get_query_vector(text):
    query_tokens = clip_processor(text=[text], return_tensors='pt')
    query_vector = clip_model.get_text_features(query_tokens['input_ids'].to(model_device))
    query_vector = F.normalize(query_vector)
    query_vector = query_vector.cpu().detach().numpy().reshape(-1)
    return query_vector

In [4]:
from sklearn.metrics import average_precision_score

In [5]:
y_true = get_y_true(df, 'banana')
query_vector = get_query_vector('a photo of a banana')
image_scores = image_vectors @ query_vector
average_precision_score(y_true, image_scores)

0.8783107024897995

In [7]:
# for each label, we will pick a few random 'train' images we will use for querying, and the rest will be used for measuring performance
np.random.seed(0)

In [8]:
df = df.assign(random_id=np.random.permutation(df.shape[0]))
df = df.assign(group_rank=df.groupby('label')['random_id'].rank(method='first').astype('int'))
df = df.assign(split=df.group_rank.apply(lambda x: 'query' if x <= 5 else 'test'))

In [9]:
# now, we will compute the average precision for each train example
query_df = df[df.split == 'query']
image_query_vecs = image_vectors[query_df.index.values]

test_df = df[df.split == 'test']
test_vec_db = image_vectors[test_df.index.values]

ntrain = 1000
random_sample = np.random.permutation(test_vec_db.shape[0])[:ntrain] # take a random sample of 1000 images
Xneg = test_vec_db[random_sample]
yneg = np.zeros(ntrain)

In [10]:
def make_query(label : str) -> str:
    remove_under = label.replace('_', ' ')
    return f'a photo of a {remove_under}'

In [23]:
def aps_from_text_query():
    aps = []
    for (idx, row), query_vector in tqdm(zip(query_df.iterrows(), image_query_vecs), total=query_df.shape[0]):
        query_vector = get_query_vector(make_query(row.label))
        image_scores = test_vec_db @ query_vector
        y_true = get_y_true(test_df, row.label)
        ap = average_precision_score(y_true, image_scores)
        aps.append(ap)
    return np.array(aps)

In [24]:
text_ap = aps_from_text_query()

100%|██████████| 1565/1565 [00:49<00:00, 31.57it/s]


In [25]:
def aps_from_knn():
    aps = []
    for (idx, row), query_vector in tqdm(zip(query_df.iterrows(), image_query_vecs), total=query_df.shape[0]):
        #    query_vector = get_query_vector(row.text)
        image_scores = test_vec_db @ query_vector
        y_true = get_y_true(test_df, row.label)
        ap = average_precision_score(y_true, image_scores)
        aps.append(ap)
    return np.array(aps)

In [26]:
knn_ap = aps_from_knn()

100%|██████████| 1565/1565 [00:23<00:00, 66.37it/s]


In [27]:
#query_df = query_df.assign(ap=aps)

In [31]:
query_df = query_df.assign(text_ap=text_ap, knn_ap=knn_ap)

In [32]:
query_df

,label,path,random_id,group_rank,split,text_ap,knn_ap
110,air_freshener,air_freshener/912a742e5b16421.png,2079,3,query,0.037112,0.011462
132,air_freshener,air_freshener/af2cd7fef2f8477.png,913,1,query,0.037112,0.043908
144,air_freshener,air_freshener/c175510563a849f.png,2369,4,query,0.037112,0.013797
166,air_freshener,air_freshener/e5f1811779644f8.png,2953,5,query,0.037112,0.048782
174,air_freshener,air_freshener/f30f586d53d9490.png,1734,2,query,0.037112,0.026023
...,...,...,...,...,...,...,...
50077,ziploc_bag,ziploc_bag/02ebd941b75048c.png,469,1,query,0.421889,0.011251
50086,ziploc_bag,ziploc_bag/0d66b8101246457.png,1913,5,query,0.421889,0.211507
50162,ziploc_bag,ziploc_bag/67c864b8a64a4af.png,973,3,query,0.421889,0.046303
50233,ziploc_bag,ziploc_bag/cd885ba732ab4c8.png,797,2,query,0.421889,0.027874


In [34]:
from sklearn import svm

def get_svm_ap():
    aps = []
    for (idx, row), query_vector in tqdm(zip(query_df.iterrows(), image_query_vecs), total=query_df.shape[0]):
        clf = svm.LinearSVC(class_weight='balanced', verbose=False, max_iter=10000, tol=1e-6, C=0.1)
        Xpos = query_vector.reshape(1, -1)
        X = np.concatenate([Xpos, Xneg], axis=0)
        y = np.concatenate([np.ones(1), np.zeros(Xneg.shape[0])])
        clf.fit(X, y) # train

        image_scores = clf.decision_function(test_vec_db)
        y_true = get_y_true(test_df, row.label)
        ap = average_precision_score(y_true, image_scores)
        aps.append(ap)

    return np.array(aps)

In [35]:
svm_ap = get_svm_ap()

100%|██████████| 1565/1565 [01:48<00:00, 14.38it/s]


In [37]:
query_df = query_df.assign(svm_ap=svm_ap)

In [100]:
from sklearn import linear_model
aps = []

for (idx, row), query_vector in tqdm(zip(query_df.iterrows(), image_query_vecs), total=query_df.shape[0]):
    #    query_vector = get_query_vector(row.text)
    clf = linear_model.LogisticRegression(class_weight='balanced', fit_intercept=True, verbose=False, max_iter=10000, tol=1e-6, C=0.1)
    Xpos = query_vector.reshape(1, -1)
    X = np.concatenate([Xpos, Xneg], axis=0)
    y = np.concatenate([np.ones(1), np.zeros(Xneg.shape[0])])
    clf.fit(X, y) # train

    image_scores = clf.decision_function(test_vec_db)
    y_true = get_y_true(test_df, row.label)
    ap = average_precision_score(y_true, image_scores)
    aps.append(ap)

100%|██████████| 1565/1565 [03:34<00:00,  7.31it/s]


In [16]:
import importlib
import playground


<module 'playground.logistic_regression' from '/Users/orm/repos/playground/playground/logistic_regression.py'>

In [42]:
torch.maximum(torch.tensor(0.), torch.tensor([-1., 0., 1.]))

tensor([0., 0., 1.])

In [73]:
from sklearn.datasets import load_iris

In [74]:
X,y = load_iris(return_X_y=True)

In [109]:
import playground.basic_trainer
import playground.logistic_regression

importlib.reload(playground.logistic_regression)
importlib.reload(playground.basic_trainer)
from playground.logistic_regression import LinearModel

In [77]:
clf = LinearModel(class_weight='balanced', label_loss_type='hinge_squared_loss', reg_norm_lambda=10., verbose=False)

In [78]:
clf2 = svm.LinearSVC(class_weight='balanced', verbose=False, max_iter=10000, tol=1e-6, C=0.1)

In [85]:
ys = (y == 2).astype('float')

In [94]:
X = X.astype('float32')

In [95]:
clf.fit(X,ys)

[{'k': 'total_loss',
  'loss': 492.4753375472613,
  'grad_norm': 1010.6362915039062},
 {'k': 'total_loss',
  'loss': 212.23198399093678,
  'grad_norm': 140.1540985107422},
 {'k': 'total_loss',
  'loss': 169.39621252645577,
  'grad_norm': 119.19063568115234},
 {'k': 'total_loss',
  'loss': 109.26077046315245,
  'grad_norm': 226.56996154785156},
 {'k': 'total_loss',
  'loss': 95.10838870593969,
  'grad_norm': 123.43391418457031},
 {'k': 'total_loss',
  'loss': 80.21454797855716,
  'grad_norm': 37.132747650146484},
 {'k': 'total_loss',
  'loss': 69.63078859392593,
  'grad_norm': 24.307775497436523},
 {'k': 'total_loss',
  'loss': 56.410178121361014,
  'grad_norm': 21.143489837646484},
 {'k': 'total_loss',
  'loss': 47.40404180311625,
  'grad_norm': 23.538684844970703},
 {'k': 'total_loss',
  'loss': 45.194081021262576,
  'grad_norm': 19.654163360595703},
 {'k': 'total_loss',
  'loss': 44.51854413228839,
  'grad_norm': 13.927041053771973},
 {'k': 'total_loss',
  'loss': 44.251566352470576,

In [99]:
np.bincount(y)

array([50, 50, 50])

In [100]:
y.shape[0] / (3 * np.bincount(y))

array([1., 1., 1.])

In [105]:
mys = np.array([1, 0, 0, 0, 0, 0])

In [106]:
mys.shape[0] / (2 * np.bincount(mys))

array([0.6, 3. ])

6.0

In [96]:
clf2.fit(X,ys)

LinearSVC(C=0.1, class_weight='balanced', max_iter=10000, tol=1e-06,
          verbose=False)

In [97]:
average_precision_score(ys, clf.decision_function(X))

0.9970782542372485

In [98]:
average_precision_score(ys, clf2.decision_function(X))

0.9948175232429083

In [121]:
import playground.basic_trainer
import playground.logistic_regression

importlib.reload(playground.logistic_regression)
importlib.reload(playground.basic_trainer)
from playground.logistic_regression import LinearModel
aps = []

for (idx, row), query_vector in tqdm(zip(query_df.iterrows(), image_query_vecs), total=query_df.shape[0]):
    # query_vector = get_query_vector(make_query(row.label))
    clf = LinearModel(class_weight='balanced', label_loss_type='hinge_squared_loss', reg_norm_lambda=10., verbose=False)
    Xpos = query_vector.reshape(1, -1)
    X = np.concatenate([Xpos, Xneg], axis=0)
    y = np.concatenate([np.ones(1), np.zeros(Xneg.shape[0])])
    clf.fit(X, y) # train
    image_scores = clf.decision_function(test_vec_db)
    y_true = get_y_true(test_df, row.label)
    ap = average_precision_score(y_true, image_scores)
    aps.append(ap)

100%|██████████| 1565/1565 [01:30<00:00, 17.24it/s]


In [120]:
ap

0.011702599291410355

In [112]:
clf2 = svm.LinearSVC(class_weight='balanced', verbose=False, max_iter=10000, tol=1e-6, C=0.1)

In [113]:
clf2.fit(X,y)

LinearSVC(C=0.1, class_weight='balanced', max_iter=10000, tol=1e-06,
          verbose=False)

In [114]:
ap

0.011687032435414848

In [115]:
image_scores2 = clf2.decision_function(test_vec_db)
ap2 = average_precision_score(y_true, image_scores2)


In [116]:
ap2

0.011673648442819949

In [68]:
query_df

,label,path,random_id,group_rank,split,text_ap,knn_ap,reg_ap,svm_ap
110,air_freshener,air_freshener/912a742e5b16421.png,2079,3,query,0.037112,0.011462,0.032078,0.011674
132,air_freshener,air_freshener/af2cd7fef2f8477.png,913,1,query,0.037112,0.043908,0.032065,0.053525
144,air_freshener,air_freshener/c175510563a849f.png,2369,4,query,0.037112,0.013797,0.032078,0.016202
166,air_freshener,air_freshener/e5f1811779644f8.png,2953,5,query,0.037112,0.048782,0.032078,0.063500
174,air_freshener,air_freshener/f30f586d53d9490.png,1734,2,query,0.037112,0.026023,0.032078,0.033975
...,...,...,...,...,...,...,...,...,...
50077,ziploc_bag,ziploc_bag/02ebd941b75048c.png,469,1,query,0.421889,0.011251,0.416618,0.012339
50086,ziploc_bag,ziploc_bag/0d66b8101246457.png,1913,5,query,0.421889,0.211507,0.416618,0.271601
50162,ziploc_bag,ziploc_bag/67c864b8a64a4af.png,973,3,query,0.421889,0.046303,0.416618,0.048199
50233,ziploc_bag,ziploc_bag/cd885ba732ab4c8.png,797,2,query,0.421889,0.027874,0.416618,0.039746


In [122]:
query_df = query_df.assign(svm_squared_ap=aps)
totals = query_df.groupby('label')[['knn_ap', 'text_ap', 'reg_ap', 'svm_ap', 'svm2_ap', 'svm_squared_ap']].mean()
totals.mean()

knn_ap            0.076140
text_ap           0.232334
reg_ap            0.233089
svm_ap            0.088182
svm2_ap           0.055714
svm_squared_ap    0.087306
dtype: float64

In [38]:
query_df = query_df.assign(reg_ap=aps)
totals = query_df.groupby('label')[['knn_ap', 'text_ap', 'reg_ap', 'svm_ap']].mean()
totals.mean()

knn_ap     0.076140
text_ap    0.232334
reg_ap     0.233089
svm_ap     0.088182
dtype: float64

In [99]:
query_df = query_df.assign(lrap=aps)
totals = query_df.groupby('label')[['ap', 'svap', 'lrap']].mean()
totals.mean()

ap      0.076140
svap    0.088269
lrap    0.078470
dtype: float64

In [101]:
query_df = query_df.assign(lrap=aps)
totals = query_df.groupby('label')[['ap', 'svap', 'lrap']].mean()
totals.mean()

ap      0.076140
svap    0.088269
lrap    0.079271
dtype: float64